In [1]:
import numpy as np
import pandas as pd

In [7]:
data= pd.read_csv('C:\\Users\\Admin\\Downloads\\DW_Retail dataset\\DW_Retail dataset\\merged_data.csv')
data.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id,aisle,department
0,2,33120,1,1,202279,prior,3,5,9,8.0,Organic Egg Whites,86,16,eggs,dairy eggs
1,2,28985,2,1,202279,prior,3,5,9,8.0,Michigan Organic Kale,83,4,fresh vegetables,produce
2,2,9327,3,0,202279,prior,3,5,9,8.0,Garlic Powder,104,13,spices seasonings,pantry
3,2,45918,4,1,202279,prior,3,5,9,8.0,Coconut Butter,19,13,oils vinegars,pantry
4,2,30035,5,0,202279,prior,3,5,9,8.0,Natural Sweetener,17,13,baking ingredients,pantry


In [9]:
# Filter for add_to_cart_order < 5
df = data[data['add_to_cart_order'] < 5]

In [11]:

# Encode users and products to integer indices for matrix representation
user_ids = df['user_id'].unique()
product_ids = df['product_id'].unique()
user_map = {id: idx for idx, id in enumerate(user_ids)}
product_map = {id: idx for idx, id in enumerate(product_ids)}
df['user_idx'] = df['user_id'].map(user_map)
df['product_idx'] = df['product_id'].map(product_map)

C:\Users\Admin\AppData\Local\Temp\ipykernel_23696\1925879585.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['user_idx'] = df['user_id'].map(user_map)
C:\Users\Admin\AppData\Local\Temp\ipykernel_23696\1925879585.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['product_idx'] = df['product_id'].map(product_map)


In [15]:
df['user_idx'].head(),df['product_idx'].head()

(0    0
 1    0
 2    0
 3    0
 9    1
 Name: user_idx, dtype: int64,
 0    0
 1    1
 2    2
 3    3
 9    4
 Name: product_idx, dtype: int64)

In [17]:
# Define model parameters
n_users, n_items = len(user_ids), len(product_ids)
n_factors = 10  # Number of latent factors
lr = 0.01  # Learning rate
n_epochs = 50  # Number of training epochs
reg = 0.01  # Regularization term to prevent overfitting

# Initialize user and item matrices with random values
user_matrix = np.random.normal(scale=1./n_factors, size=(n_users, n_factors))
item_matrix = np.random.normal(scale=1./n_factors, size=(n_items, n_factors))

# Training with Stochastic Gradient Descent (SGD)
for epoch in range(n_epochs):
    for _, row in df.iterrows():
        u = row['user_idx']
        i = row['product_idx']
        rating = row['add_to_cart_order']
        
        # Predicted rating
        pred = np.dot(user_matrix[u], item_matrix[i])
        
        # Error
        error = rating - pred
        
        # Update user and item latent factors
        user_matrix[u] += lr * (error * item_matrix[i] - reg * user_matrix[u])
        item_matrix[i] += lr * (error * user_matrix[u] - reg * item_matrix[i])
    
    # Calculate loss
    loss = 0
    for _, row in df.iterrows():
        u = row['user_idx']
        i = row['product_idx']
        rating = row['add_to_cart_order']
        pred = np.dot(user_matrix[u], item_matrix[i])
        loss += (rating - pred) ** 2 + reg * (np.linalg.norm(user_matrix[u]) + np.linalg.norm(item_matrix[i]))
    
    print(f"Epoch {epoch+1}/{n_epochs}, Loss: {loss}")




C:\Users\Admin\AppData\Local\Temp\ipykernel_23696\722300811.py:27: RuntimeWarning: overflow encountered in multiply
  item_matrix[i] += lr * (error * user_matrix[u] - reg * item_matrix[i])
C:\Users\Admin\AppData\Local\Temp\ipykernel_23696\722300811.py:26: RuntimeWarning: overflow encountered in multiply
  user_matrix[u] += lr * (error * item_matrix[i] - reg * user_matrix[u])


Epoch 1/50, Loss: nan
Epoch 2/50, Loss: nan


KeyboardInterrupt: 

In [19]:
# Function to predict for a given user and product
def predict(user_id, product_id):
    u = user_map.get(user_id)
    i = product_map.get(product_id)
    if u is None or i is None:
        return "User or product not found."
    return np.dot(user_matrix[u], item_matrix[i])

# Example prediction for a specific user and product
user_id = 41658
product_id = 9550
prediction = predict(user_id, product_id)
print(f"Prediction (user {user_id} for product {product_id}): {prediction}")

# Generate recommendations
def recommend(user_id, n_recommendations=5):
    u = user_map.get(user_id)
    if u is None:
        return "User not found."
    scores = np.dot(user_matrix[u], item_matrix.T)
    product_indices = np.argsort(-scores)[:n_recommendations]
    recommended_products = [product_ids[i] for i in product_indices]
    return recommended_products

# Example recommendation for a specific user
user_id = 41658
recommendations = recommend(user_id, n_recommendations=3)
print(f"Top recommendations for user {user_id}: {recommendations}")

Prediction (user 41658 for product 9550): nan
Top recommendations for user 41658: [33120, 19947, 7573]
